In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

from env import get_connection

## Exercises

The end result of this exercise should be a Jupyter notebook named `model`.

Using [saas.csv](https://ds.codeup.com/saas.csv) or log data from API usage

1. Split data (train/test) and resample by any period, except daily, and aggregate using the sum. 

In [2]:
df = pd.read_csv("saas.csv", parse_dates=['Month_Invoiced'], index_col='Month_Invoiced').sort_index()

In [3]:
df.isna().sum()

Customer_Id          0
Invoice_Id           0
Subscription_Type    0
Amount               0
dtype: int64

In [4]:
#, usecols=['Month_Invoiced', 'Subscription_Type', 'Amount']

In [5]:
train_index = int(len(df) * 0.6)

In [6]:
val_index = int(len(df) * 0.85)

In [7]:
train = df[:train_index]

In [8]:
train.shape

(474274, 4)

In [9]:
val = df[train_index:val_index]

In [10]:
val.shape

(197615, 4)

In [11]:
test = df[val_index:]

In [12]:
test.shape

(118569, 4)

In [13]:
df.shape[0] == train.shape[0] + val.shape[0] + test.shape[0]

True

In [14]:
train.resample('W').sum()

,Customer_Id,Invoice_Id,Subscription_Type,Amount
Month_Invoiced,,,,
2014-02-02,20199990000,100199990000,7170.0,35850.0
2014-02-09,0,0,0.0,0.0
2014-02-16,0,0,0.0,0.0
2014-02-23,0,0,0.0,0.0
2014-03-02,20000052061,99592010100,7438.0,37190.0
...,...,...,...,...
2016-03-06,16230363306,87400384855,10391.0,51955.0
2016-03-13,0,0,0.0,0.0
2016-03-20,0,0,0.0,0.0


2. Forecast, plot and evaluate using each at least 4 of the methods we discussed:
    - Last Observed Value
    - Simple Average
    - Moving Average
    - Holt's Linear Trend 
    - Holt's Seasonal Trend
    - Based on previous year/month/etc., this is up to you.

### Last Observed Value

In [15]:
train['Subscription_Type'][-1:][0]

0.0

In [16]:
len(train[train['Subscription_Type'] == 0.0])

320037

In [17]:
len(train[train['Subscription_Type'] != 0.0])

154237

In [18]:
len(train.Customer_Id.value_counts())

25000

In [19]:
len(train.Invoice_Id.value_counts())

474274

In [20]:
last_subscription = train['Subscription_Type'].tail(1)

In [21]:
last_amount = train['Amount'].tail(1)

In [22]:
yhat_df = pd.DataFrame(
    {'subscription_total': last_subscription,
     'amount': last_amount},
    index=val.index)

#rmse0 = round(sqrt(mean_squared_error(val['Subscription_Type'], yhat_df['subscription_total'])), 0)

rmse1 = round(sqrt(mean_squared_error(val['Amount'], yhat_df['amount'])), 0)

Basically the prediction is 0.0 for Subscription Type and Amount.

### Simple Average

In [23]:
avg_subscriptions = round(train['Subscription_Type'].mean(), 2)
avg_amount = round(train['Amount'].mean(), 2)
avg_subscriptions, avg_amount

(0.52, 2.6)

In [26]:
def make_baseline_predictions(sales_predictions=None, quantity_predictions=None):
    yhat_df = pd.DataFrame({'Subscription_Type': avg_subscriptions,
                           'Amount': avg_amount},
                          index=val.index)
    return yhat_df

In [27]:
yhat_df = make_baseline_predictions(avg_subscriptions, avg_amount)

In [28]:
yhat_df.head()

,Subscription_Type,Amount
Month_Invoiced,,
2016-03-31,0.52,2.6
2016-03-31,0.52,2.6
2016-03-31,0.52,2.6
2016-03-31,0.52,2.6
2016-03-31,0.52,2.6


In [29]:
rmse0 = round(sqrt(mean_squared_error(val['Subscription_Type'], yhat_df['Subscription_Type'])), 0)
rmse0

1.0

In [30]:
rmse1 = round(sqrt(mean_squared_error(val['Amount'], yhat_df['Amount'])), 0)
rmse1

5.0

### Moving Average

Bonus: 
1. Using the store item demand data, create a forecast of `sales_total` and `quantity` for 2018 using the `Previous Cycle` approach.  .  
2. Predict 2018 total **monthly** sales for a single store and/or item by creating a model using prophet.
3. Return a dataframe with the month, store_id, y-hat, and the confidence intervals (y-hat lower, y-hat upper).
4. Plot the 2018 monthly sales predictions.